In [16]:

from minio import Minio
from minio.error import S3Error
import os
from dotenv import load_dotenv

load_dotenv()


minio_endpoint = "localhost:9000"
access_key =os.getenv('MINIO_ACCESS_KEY')
secret_key = os.getenv('MINIO_SECRET')
bucket_name = os.getenv('MINIO_BUCKET_NAME')

In [15]:
# def create_minio_client(endpoint, access_key, secret_key, secure=False):
#     return Minio(endpoint, access_key=access_key, secret_key=secret_key, secure=secure)
def create_minio_client(endpoint, access_key, secret_key, secure=False):
    client = Minio(endpoint, access_key=access_key, secret_key=secret_key, secure=secure)
    print(f"Created Minio client: {client}")
    return client

def ensure_bucket(client, bucket_name):
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
        print(f"Created bucket: {bucket_name}")

In [17]:
# client = Minio('localhost:9000', access_key=access_key, secret_key=secret_key, secure=False)
# client



def create_minio_client():
    client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=False)
    return client

In [18]:
conn = create_minio_client()
conn

In [21]:
#MINIO_ENDPOINT = "minio:9000"


#client = create_minio_client(MINIO_ENDPOINT,access_key, secret_key)
bucket_name = "rogerlake"
ensure_bucket(conn, bucket_name)


In [ ]:

# def run_spark_job():
#     """Dynamically fetch the latest file and run Spark transformation."""
#     client = create_minio_client()
#     latest_raw_file = get_latest_raw_file(client)
#     processed_parquet_file = get_processed_parquet_path()

#     spark_command = [
#         "docker", "exec", "healthstream-spark-master",  # Remove the "-it" flag
#         "/opt/bitnami/spark/bin/spark-submit",
#         "--packages", "org.apache.hadoop:hadoop-aws:3.2.0",
#         "--master", "spark://healthstream-spark-master:7077",
#         "--executor-memory", "4g",
#         "--driver-memory", "2g",
#         "/opt/spark/jobs/transform_health_data.py",
#         f"s3a://{bucket_name}/{latest_raw_file}",
#         f"s3a://{bucket_name}/{processed_parquet_file}"
#     ]
#     print("Running Spark job......")
    
#     # For better debugging, capture output
#     result = subprocess.run(
#         spark_command,
#         check=False,  # Change to False to avoid exceptions
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True
#     )
    
#     # Print output for debugging
#     print(f"Return code: {result.returncode}")
#     print(f"STDOUT: {result.stdout}")
#     print(f"STDERR: {result.stderr}")
    
#     # Now raise exception if needed
#     if result.returncode != 0:
#         raise Exception(f"Spark job failed with return code {result.returncode}")

# # Task to run Spark job
# spark_transform_task = PythonOperator(
#     task_id="run_spark_transformation",
#     python_callable=run_spark_job,
#     dag=dag,
# )

latest_raw_file = get_latest_raw_file(client)
processed_parquet_file = get_processed_parquet_path()

# spark_transform_task = SparkSubmitOperator(
#     task_id='spark_transform',
#     application='/opt/spark/jobs/transform_health_data.py',
#     conn_id='spark_default',
#     packages='org.apache.hadoop:hadoop-aws:3.2.0',
#     env_vars={
#         'JAVA_HOME': '/usr/lib/jvm/java-17-openjdk-amd64'
#     },
#     conf={
#         'spark.master': 'spark://healthstream-spark-master:7077',
#         'spark.executor.memory': '4g',
#         'spark.driver.memory': '2g'
#     },
#     application_args=[
#         f"s3a://{bucket_name}/{latest_raw_file}",
#         f"s3a://{bucket_name}/{processed_parquet_file}"
#     ],
#     dag=dag
# )


In [ ]:

FROM apache/airflow:2.10.5

# Switch to root for initial setup
USER root

# Install system dependencies

RUN apt-get update && \
    apt-get install -y --no-install-recommends \
    build-essential \
    default-jdk \
    git \
    curl \
    unzip \
    && rm -rf /var/lib/apt/lists/*

# Set Java environment variables
ENV JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
ENV PATH=$JAVA_HOME/bin:$PATH


# Copy requirements
COPY ../requirements.txt /requirements.txt

# Ensure correct file ownership
RUN chown airflow:root /requirements.txt

# Switch to airflow user
USER airflow

# Install dependencies
RUN pip install --upgrade pip && \
    pip install -r /requirements.txt

# # We'll install faker directly since we can't access generator folder at build time
RUN pip install faker
RUN pip install tqdm
RUN pip install minio
RUN pip install apache-airflow-providers-apache-spark

# Set PYTHONPATH
#ENV PYTHONPATH="/opt/airflow:/opt/airflow/dags:/opt/airflow/generator"
ENV PYTHONPATH="/opt/airflow:/opt/airflow/dags:/opt/airflow/generator"


WORKDIR /opt/airflow

In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.providers.apache.spark.operators.spark_submit import SparkSubmitOperator
from airflow.operators.python import PythonOperator
import os
from datetime import datetime
import sys
from pathlib import Path
from minio import Minio
import subprocess
from dotenv import load_dotenv
from airflow.models import Variable
from pyspark.sql import SparkSession

load_dotenv()

bucket_name = "rogerlake"
# minio_endpoint = os.getenv("MINIO_ENDPOINT", "172.18.0.2:9000")
minio_endpoint = Variable.get("MINIO_ENDPOINT")
access_key = Variable.get("MINIO_ACCESS_KEY")
print(f"Using access", access_key)
secret_key = Variable.get("MINIO_SECRET")
print(f"------Connecting to MinIO at: {minio_endpoint}")
print(f"Using bucket: {bucket_name}")



# Ensure Python can find the 'generator' module
sys.path.append('/opt/airflow')
sys.path.append('/opt/airflow/generator')

from generator.minio_client import create_minio_client
# Define environment variables
RAW_PREFIX = "health_records/_raw_data_/health_record_raw_"
PROCESSED_PREFIX = "health_records/_cleaned_/health_data_"

client = create_minio_client()
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
parquet_object = "health_records/_cleaned_/health_data_{timestamp}.parquet"



def get_latest_raw_file(minio_client):
    """Retrieve the latest raw data file from MinIO."""
    raw_files = list(minio_client.list_objects(bucket_name, prefix=RAW_PREFIX))
    if not raw_files:
        raise FileNotFoundError(" No raw files found in MinIO!")
    # Extract timestamps from filenames
    latest_file = max(raw_files, key=lambda x: x.last_modified)
    return latest_file.object_name

# Function to generate processed file path with timestamp
def get_processed_parquet_path():
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    return f"{PROCESSED_PREFIX}{timestamp}.parquet"

# latest_raw_file = get_latest_raw_file(client)
# processed_parquet_file = get_processed_parquet_path()

# Default args for the DAG
default_args = {
    "owner": "airflow",
    "depends_on_past": False,
    "start_date": datetime(2024, 1, 1),
    "email_on_failure": False,
    "retries": 1,
    "retry_delay": timedelta(minutes=5),
}

# Initialize DAG
dag = DAG(
    "health_data_etl",
    default_args=default_args,
    description="ETL pipeline for processing health records with Spark",
    schedule_interval="@daily",
    catchup=False,
    tags=["spark", "minio", "etl"],
)

# Fetch Latest Raw File
fetch_latest_raw_file_task = PythonOperator(
    task_id="fetch_latest_raw_file",
    python_callable=get_latest_raw_file(client),
    do_xcom_push=True,  # Save result in XCom for later use
    dag=dag
)

# Generate Processed File Path
generate_parquet_path_task = PythonOperator(
    task_id="generate_parquet_path",
    python_callable=get_processed_parquet_path,
    do_xcom_push=True,  # Save result in XCom for later use
    dag=dag
)

#  Run Spark Job

spark_transform_task = SparkSubmitOperator(
    task_id='spark_transform',
    application='/opt/spark/jobs/transform_health_data.py',
    conn_id='spark_default',  # Ensure this is correctly configured in Airflow Connections
    packages='org.apache.hadoop:hadoop-aws:3.2.0',
    env_vars={
        'JAVA_HOME': '/usr/lib/jvm/java-17-openjdk-amd64',
        'SPARK_HOME': '/opt/spark',
        'PATH':  '/opt/spark/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', #
        'PYTHONPATH': '/opt/spark/jobs:/opt/spark/python:/opt/spark/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH'
        
    },
    conf={
        'spark.master': 'spark://healthstream-spark-master:7077',
        'spark.executor.memory': '4g',
        'spark.driver.memory': '2g',
        "spark.submit.deployMode": "client",
        'spark.executor.extraJavaOptions': '-Dfile.encoding=UTF-8',
        'spark.driver.extraJavaOptions': '-Dfile.encoding=UTF-8',
        'spark.hadoop.fs.s3a.path.style.access': 'true',
        'spark.hadoop.fs.s3a.connection.ssl.enabled': 'false',
        'spark.files': '/opt/spark/jobs/transform_health_data.py'
        #'spark.jars': '/opt/bitnami/spark/jars/hadoop-aws-3.3.2.jar,/opt/bitnami/spark/jars/aws-java-sdk-bundle-1.11.1026.jar'
    },
    application_args=[
        "{{ ti.xcom_pull(task_ids='fetch_latest_raw_file') }}",
        "{{ ti.xcom_pull(task_ids='generate_parquet_path') }}"
    ],
    # application_args=[
    #     f"s3a://{bucket_name}/{latest_raw_file}",
    #     f"s3a://{bucket_name}/{processed_parquet_file}"
    # ],
    verbose=True,
    dag=dag
)


#  Verify Output
def verify_output():
    """Check if Parquet file exists in MinIO."""
    if client.stat_object(bucket_name, parquet_object):
        print(f" Processed data available: {parquet_object}")
    else:
        raise FileNotFoundError(" Parquet file not found!")

verify_output_task = PythonOperator(
    task_id="verify_parquet_output",
    python_callable=verify_output,
    dag=dag,
)

# Task Dependencies
fetch_latest_raw_file_task >> generate_parquet_path_task >> spark_transform_task >> verify_output_task
# spark_transform_task >> verify_output_task


In [ ]:
def main():
    """Main function to orchestrate the ETL process."""
    parser = argparse.ArgumentParser(description='Transform health data from CSV to Parquet')
    parser.add_argument('input_path', help='Input S3 path (can be auto-detected if not provided)',
                        nargs='?', default=None)
    parser.add_argument('output_path', help='Output S3 path for Parquet files',
                        nargs='?', default=None)
    parser.add_argument('--bucket', help='MinIO/S3 bucket name', 
                        default='healthdata')
    
    args = parser.parse_args()
    
    
    # Initialize clients
    spark = initialize_spark()
    client = create_minio_client()
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    
    try:
        # Determine input path
        if args.input_path:
            input_path = args.input_path
        else:
            # Auto-detect latest file
            latest_object = get_latest_file(client, bucket_name, "_raw_data_")
            input_path = f"s3a://{bucket_name}/{latest_object}"
        
        # Determine output path
        if args.output_path:
            output_path = args.output_path
        else:
            output_path = f"s3a://{bucket_name}/cleaned_data/health_record_{timestamp}.parquet"
        
        print(f"Processing: {input_path} -> {output_path}")
        
        # Clean and transform data
        cleaned_df = clean_transform_data(spark, input_path)
        
        # Save to parquet
        save_to_parquet(cleaned_df, output_path)
        
        print("Data transformation completed successfully!")
        
    except Exception as e:
        print(f"Error in ETL process: {e}")
        raise
    finally:
        # Clean up
        spark.stop()


if __name__ == "__main__":
    main()